In [338]:
import pandas as pd
import numpy as np

## importamos la función
from Limpieza_res_redatam import conversion_redatam_matriz

In [335]:
ruta_l=r"C:\Users\Guillermo\Desktop\Python\05. Lib gestión pública"
### abrimos dos ejemplos de resultados de Redatam
column=["info_g","vars","fre","por", "cum"]
df=pd.read_excel(ruta_l+"\\Bases originales\\identi_INEI.xlsx", names=column) ## identificación etnica
df2=pd.read_excel(ruta_l+"\\Bases originales\\identi_INEI_pro.xlsx", names=column) ## tiene DNI

#### Revisamos la documentación de la función

In [337]:
conversion_redatam_matriz?

Signature: conversion_redatam_matriz(df)
Docstring:
conversion_redatam_matriz(
df: 'DataFrame') -> Dataframe,

Docstring:
Permite de organizar los datos descargados de la plataforma de REDATAM - (INEI) en una formato de base datos de dos dimensiones.
La base de datos de ingreso corresponde a un archivo en formato Excel, el cual se descarga luego de la busqueda de variables en REDATAM
La base de datos resultante es un DataFrame con el ubigeo como índice
File:      c:\users\guillermo\desktop\python\05. lib gestión pública\limpieza_res_redatam.py
Type:      function


#### Identificación etnica - Nivel: distrito

In [331]:
df1=conversion_redatam_matriz(df)
df1.head()

vars,Aimara,Blanco,Mestizo,Nativo o indígena de la amazonía,"Negro, moreno, zambo, mulato / pueblo afroperuano o afrodescendiente",Nikkei,No sabe / No responde,Otro,Parte de otro pueblo indígena u originario,Quechua,Tusán,Total
ubigeo,,,,,,,,,,,,
010101,23,1498,19867,77,791,3,598,188,28,1606,1,24680
010102,0,3,119,0,0,0,11,0,0,76,0,209
010103,0,16,745,0,4,0,14,4,0,3,0,786
010104,0,2,493,0,4,0,10,0,0,14,0,523
010105,0,5,440,0,9,0,3,1,0,1,0,459


In [332]:
%%timeit
conversion_redatam_matriz(df)

518 ms ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cuenta con DNI - Nivel: Provincia

In [336]:
df_p=conversion_redatam_matriz(df2)
df_p.head()

vars,No recuerda,No tiene documento alguno,Solo tiene carné de extranjería,Solo tiene partida de nacimiento,Tiene DNI,Total
ubigeo,,,,,,
0101,4431,127,54,202,50692,55506
0102,6170,1372,40,1231,65287,74100
0103,2120,78,27,116,23296,25637
0104,4868,1863,16,1517,34206,42470
0105,3687,277,17,290,40165,44436
